In [167]:
print("hello world")

hello world


In [168]:
import pandas as pd
import os

In [169]:
file_path = 'data/mini/NL RIVM-NEVO_mini.txt'

with open(file_path, 'r') as file:
    lines = file.readlines()

headers = lines[0].strip().split('\t')

data = []
for line in lines[1:]:
    fields = line.strip().split('\t')
    if len(fields) < len(headers):
        fields += [''] * (len(headers) - len(fields))
    data.append(fields)

# print(data)

df_nevo_langal = pd.DataFrame(data, columns=headers)

def convert_to_set(langual_codes):
    return set(langual_codes.split())

# Apply the function to the LANGUALCODES column
df_nevo_langal['LANGUALCODES'] = df_nevo_langal['LANGUALCODES'].apply(convert_to_set)

df_nevo_langal.head()

,FOODID,ORIGFDNM,ENGFDNAM,LANGUALCODES,REMARKS
0,945,Aardappel driekantjes bereid in diamantvet,Potatoes triangulardeep fat fried,"{B1218, A0152, A0706, A0829}",
1,947,Aardappel driekantjes bereid in oven,Potatoes triangular prepared in oven,"{B1218, A0152, A0706, A0829}",
2,946,Aardappel driekantjes bereid in zonnebloemolie,Potatoes triangular prep in sunflower oi,"{B1218, A0152, A0706, A0829}",
3,1457,Aardappelen gebakken,Potatoes fried,"{P0078, F0014, K0003, A0829, G0026, P0024, A01...",
4,982,Aardappelen gekookt,Potatoes boiled,"{P0078, F0014, G0015, K0003, A0829, P0024, A01...",


In [170]:
file_path = 'data/mini/DK DTU_FDB 2008-06-26.txt'

with open(file_path, 'r') as file:
    lines = file.readlines()

headers = lines[0].strip().split('\t')

data = []
for line in lines[1:]:
    fields = line.strip().split('\t')
    if len(fields) < len(headers):
        fields += [''] * (len(headers) - len(fields))
    data.append(fields)

df_frida_langal = pd.DataFrame(data, columns=headers)


def convert_to_set(langual_codes):
    return set(langual_codes.split())

# Apply the function to the LANGUALCODES column
df_frida_langal['LANGUALCODES'] = df_frida_langal['LANGUALCODES'].apply(convert_to_set)

df_frida_langal.head()

,FOODID,ORIGFDNM,ENGFDNAM,LANGUALCODES,REMARKS
0,0001,"Abrikos, t�rret","Apricot, dried","{J0116, B1529, F0003, N0001, C0139, A0707, A07...",
1,0002,"Agurk, r�","Cucumber, raw","{F0003, A0676, K0003, A0731, A0705, A0629, A08...",
2,0003,"Ananas, r�","Pineapple, raw","{F0003, A0833, A0707, A0732, A0629, K0003, A01...",
3,0004,"Ananas, konserves","Pineapple, canned","{N0001, F0014, A0707, A0732, A0629, C0230, A01...",
4,0005,"Appelsin, r�","Orange, raw","{F0003, A0833, A0707, A0732, A0629, G0003, A01...",


In [ ]:
def jaccard_similarity(set1, set2):
    intersection = set1.intersection(set2)
    union = set1.union(set2)
    return len(intersection) / len(union)

def langual_similarity(code1, code2):
    set1 = set(code1.split())
    set2 = set(code2.split())
    return jaccard_similarity(set1, set2)

# EXAMPLE: calculating similarity
# similarity = jaccard_similarity(
#     df_nevo_langal['LANGUALCODES'][0],
#     df_nevo_langal['LANGUALCODES'][1]
# )


In [171]:
def jaccard_similarity(set1, set2):
    intersection = set1.intersection(set2)
    union = set1.union(set2)
    return len(intersection) / len(union)

def langual_similarity(code1, code2):
    set1 = set(code1.split())
    set2 = set(code2.split())
    return jaccard_similarity(set1, set2)

# EXAMPLE: calculating similarity
# similarity = jaccard_similarity(
#     df_nevo_langal['LANGUALCODES'][0],
#     df_nevo_langal['LANGUALCODES'][1]
# )

# Calculate Jaccard similarity between the first row and the other rows
first_row_set = df_nevo_langal['LANGUALCODES'][0]
df_nevo_langal['SIMILARITYSCORE'] = df_nevo_langal['LANGUALCODES'].apply(lambda x: jaccard_similarity(first_row_set, x))

# Set pandas options to display all rows
pd.set_option('display.max_rows', None)

# Sort the DataFrame based on SIMILARITY_TO_FIRST column from greatest to least
df_nevo_langal = df_nevo_langal.sort_values(by='SIMILARITYSCORE', ascending=False)

df_nevo_langal

,FOODID,ORIGFDNM,ENGFDNAM,LANGUALCODES,REMARKS,SIMILARITYSCORE
0,945,Aardappel driekantjes bereid in diamantvet,Potatoes triangulardeep fat fried,"{B1218, A0152, A0706, A0829}",,1.000000
8,1680,Aardappelkroketten diepvries bereid,Potato croquettes frozen ready to serve,"{B1218, A0152, A0706, A0829}",,1.000000
1,947,Aardappel driekantjes bereid in oven,Potatoes triangular prepared in oven,"{B1218, A0152, A0706, A0829}",,1.000000
13,1150,Aardappelschijfjes diepvries onbereid,Potatoes sliced frozen,"{B1218, A0152, A0706, A0829}",,1.000000
12,1678,Aardappelschijfjes diepvries gefrituurd,Potatoes sliced frozen deep-fried,"{B1218, A0152, A0706, A0829}",,1.000000
10,737,Aardappelpuree instant bereid,Potatoes mashed instant ready to serve,"{B1218, A0152, A0706, A0829}",,1.000000
9,121,Aardappelpuree bereid,Potatoes mashed ready to serve,"{B1218, A0152, A0706, A0829}",,1.000000
11,1455,Aardappelpureepoeder,Potato pur�e powder,"{B1218, A0152, A0706, A0829}",,1.000000
2,946,Aardappel driekantjes bereid in zonnebloemolie,Potatoes triangular prep in sunflower oi,"{B1218, A0152, A0706, A0829}",,1.000000
7,1,Aardappelen rauw,Potatoes raw,"{F0003, G0003, K0003, A0829, P0024, A0152, C02...",,0.285714


In [172]:
# Calculate Jaccard similarity between the first row and the other rows
first_row_set = df_nevo_langal['LANGUALCODES'][0]
df_frida_langal['SIMILARITYSCORE'] = df_frida_langal['LANGUALCODES'].apply(lambda x: jaccard_similarity(first_row_set, x))

# Sort the DataFrame based on SIMILARITY_TO_FIRST column from greatest to least
df_frida_langal = df_frida_langal.sort_values(by='SIMILARITYSCORE', ascending=False)

df_frida_langal

,FOODID,ORIGFDNM,ENGFDNAM,LANGUALCODES,REMARKS,SIMILARITYSCORE
29,0030,"Voksb�nner, r�","Beans, wax, raw","{A0831, A0650, F0003, N0001, K0003, A0629, A07...",,0.047619
26,0027,"Gr�nne b�nner, dybfrost","Beans, green, frozen","{A0831, A0650, N0001, K0003, A0731, A0629, A07...",,0.047619
24,0025,"Hvide b�nner, t�rrede","Beans, white, dried","{A0831, J0116, F0003, N0001, C0133, K0003, A06...",,0.047619
27,0028,"Gr�nne b�nner, r�","Beans, green, raw","{A0831, A0650, F0003, N0001, K0003, A0731, A06...",,0.047619
22,0023,"Br�ndkarse, r�","Watercress, raw","{A0650, F0003, K0003, A0731, A0629, A0705, A08...",,0.047619
28,0029,"B�nner, brune, t�rrede","Beans, brown, dried","{A0831, J0116, F0003, N0001, C0133, K0003, A06...",,0.047619
20,0021,"Broccoli, r�","Broccoli, raw","{A0650, F0003, C0237, K0003, A0731, A0629, A07...",,0.047619
60,0061,"Gr�skar, r�","Pumpkin, raw","{A0650, F0003, K0003, A0731, A0629, A0705, A08...",,0.047619
30,0031,"Voksb�nner, konserves","Beans, wax, canned","{A0831, N0001, F0014, H0001, A0731, A0629, A07...",,0.047619
87,0088,"Hvidk�l, r�","Cabbage, white, raw","{A0650, F0003, B1406, K0003, A0731, A0629, A07...",,0.047619
